# Домашнее задание (50 баллов)

В этом домашнем задании вы познакомитесь с основами NLP, научитесь обрабатывать тексты.

В местах, где используется `...` (elipsis), требуется заменить его на код.

Установим необходимые зависимости:

In [2]:
!pip install -U pip
!pip install nltk tqdm seqeval scikit-learn datasets numpy

In [3]:
from typing import List, Dict, Tuple, Callable
from collections import Counter, defaultdict
import unicodedata
import random

import nltk
import numpy as np
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from nltk.stem.snowball import EnglishStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer




nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
print("Done")


Done


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/igorchebuniaev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/igorchebuniaev/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/igorchebuniaev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Токенизация (15 баллов)

Токенизация - это процесс преобразования текста в набор токенов.
Наивная реализация разбивает текст по пробелам. Более умные реализации учитывают пунктуацию.

### Библиотека NLTK (2 балла)

Научимся работать с токенизацией NLTK, где уже реализована работа с пунктуацией.

https://www.nltk.org/

In [4]:
def tokenize(text: str, language: str = "english") -> List[str]:
    # используйте функцию nltk.word_tokenize для разбиения текста на токены
    # https://www.nltk.org/api/nltk.tokenize.word_tokenize.html
    # raise NotImplementedError()
    return nltk.word_tokenize(text, language)


assert tokenize("") == []
assert tokenize("Hello, world!") == ["Hello", ",", "world", "!"]
assert tokenize("EU rejects German call to boycott British lamb.") == ["EU", "rejects", "German", "call", "to", "boycott", "British", "lamb", "."]
print("test_tokenize OK")


test_tokenize OK


### Нормализация (3 балла)

Добавим нормализацию после токенизации. Пробуем [лемматизацию](https://www.nltk.org/api/nltk.stem.wordnet.html#nltk.stem.wordnet.WordNetLemmatizer) , [стемминг](https://www.nltk.org/api/nltk.stem.snowball.html#nltk.stem.snowball.EnglishStemmer) и [юникод](https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize) нормализацию. Напишем функцию, которая будет принимать на вход токен после токенизации, нормализовать в NFC юникод форму, переводит в нижний регистр, лемматизирует слово и, если слово не изменилось после лемматизации, применяет стемминг.


Создайте функцию `normalize`:
   - Функция `normalize` должна принимать строку `token` и возвращать нормализованный токен.
   - Примените к токену Unicode нормализацию с помощью `unicode_nfc_normalizer`.
   - Преобразуйте токен в нижний регистр.
   - Примените лемматизацию с помощью `lemmatizer`.
   - Если лемматизированный токен отличается от исходного, верните его. В противном случае, примените стемминг с помощью `stemmer` и верните результат.

In [5]:
stemmer = EnglishStemmer()
lemmatizer = WordNetLemmatizer()
unicode_nfc_normalizer = lambda token: unicodedata.normalize("NFC", token)


def normalize(token: str) -> str:
    """
    Нормализует токен, применяя Unicode нормализацию, преобразование в нижний регистр,
    лемматизацию и стемминг при необходимости.

    :param token: Токен для нормализации
    :return: Нормализованный токен
    """
    input_token = token
    for func in [unicode_nfc_normalizer, str.lower, lemmatizer.lemmatize]:
        token = func(token)
    if token != input_token.lower():
        return token
    return stemmer.stem(token)

test_tokens = ["Worlds", "churches", "Helping"]
assert [normalize(token) for token in test_tokens] == ["world", "church", "help"]
print("test_normalize OK")


test_normalize OK


### Добавляем Словарь (10 баллов)

Современные токенайзеры не только разбивают строки на токены, но и преобразуют последовательность токенов в последовательность числел. Объединим функцию токенизации, нормализации и отображения из токенов в индексы в один объект токенайзера.

Напишите класс `Tokenizer` для токенизации и нормализации текста.

Построение словаря:
   - Создайте метод `_build_vocabulary`, который принимает список текстов `texts` и обновляет словарь токенов.
   - Для каждого текста:
     - Токенизируйте и нормализуйте текст.
     - Обновите счетчик вхождений слов.
   - После обработки всех текстов для каждого слова, которое встречается не менее `min_count` раз, добавьте слово в словарь `word2idx` и список `idx2word`.

Кодирование и декодирование:
   - Создайте метод `encode_word`, который принимает слово `word` и возвращает его индекс с применением нормализации.
   - Создайте метод `encode`, который принимает текст `text` и возвращает список индексов токенов.
   - Создайте метод `decode`, который принимает список индексов `input_ids` и возвращает текст, вставляя пробелы между токенами.

> Note: для функций, которые могут долго исполнятся (`_build_vocab`), рекомендуется использовать библиотеку tqdm.

In [6]:
class Tokenizer:
    
    def __init__(
            self, 
            texts: List[str], 
            min_count: int = 1,
    ) -> None:
        """
        Инициализация токенизатора.

        :param texts: список текстов для построения словаря
        :param tokenize_fn: функция для токенизации текста
        :param normalize_fn: функция для нормализации токенов
        :param min_count: минимальное количество вхождений слова для включения в словарь
        """
        self.min_count = min_count
        self.word2idx = {"<PAD>": 0, "<BOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.unk_token_id = 3
        self.idx2word = ["<PAD>", "<BOS>", "<EOS>", "<UNK>"]
        self.word2count = Counter()
        self._build_vocabulary(texts)

    def _build_vocabulary(self, texts: List[str]):
        """
        Построение словаря на основе списка текстов.

        :param texts: список текстов
        """
        # Проходимся по всем текстам, токенизируем текст,
        # нормализуем токены и обновляем self.word2count
        for text in tqdm(texts, desc="Считаем слова"):
            tokens = tokenize(text)
            tokens = [normalize(token) for token in tokens]
            self.word2count.update(tokens)
            
        # Теперь у нас есть заполненный self.word2count
        # ключи - нормализованные токены, значения - их встерчаемость в тексте
        # нужно добавить в словарь новый токен (обновить self.word2idx и self.idx2word), 
        # если его встречаемость не меньше self.min_count
        for key in tqdm(self.word2count, desc="Строим словарь"):
            if self.word2count[key] >= self.min_count:
                self.word2idx[key] = len(self.word2idx)
                self.idx2word.append(key)


    def encode_word(self, text: str) -> int:
        """
        Кодирование слова в индекс с применением нормализации.

        :param text: слово
        :return: индекс слова
        """
        # raise NotImplementedError()
        return self.word2idx.get(normalize(text), self.unk_token_id)

    def encode(self, text: str) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :return: набор индексов токенов
        """
        # raise NotImplementedError()
        tokens = tokenize(text)
        tokens = [normalize(token) for token in tokens]
        return [self.encode_word(token) for token in tokens]

    def decode(self, input_ids: List[int]) -> str:
        """
        Декодирование набора индексов в текст. Вставляет пробел между декодированнми токенами.

        :param input_ids: набор индексов токенов
        :return: текст
        """
        # raise NotImplementedError()
        return " ".join([self.idx2word[idx] for idx in input_ids])


    def __len__(self) -> int:
        """
        Возвращает количество уникальных токенов в словаре.

        :return: количество уникальных токенов
        """
        return len(self.word2idx)

    def __contains__(self, item: str) -> bool:
        """
        Проверяет, содержится ли слово в словаре.

        :param item: слово
        :return: True, если слово содержится в словаре, иначе False
        """
        return item in self.word2idx

    def __str__(self):
        """
        Возвращает строковое представление словаря.

        :return: строковое представление словаря
        """
        return str(self.word2idx)


corpus = ["Hello, world!", "I love Python!"]
tokenizer = Tokenizer(corpus, min_count=1)
encoded = tokenizer.encode("Hello, Python! I love you")
assert tokenizer.decode(encoded) == "hello , python ! i love <UNK>"
print("test_tokenizer OK")

Считаем слова:   0%|          | 0/2 [00:00<?, ?it/s]

Строим словарь:   0%|          | 0/7 [00:00<?, ?it/s]

test_tokenizer OK


## TF-IDF (20 баллов)


### Класс TFIDF (10 баллов)

Создайте класс `TFIDF` для вычисления TF-IDF значений.

Вам нужно реализовать следующие функции:
### add_doc
0. Увеличиваем счетчик num_docs - число документов в обучающей выборке
1. Токенизируем текст
2. Берем уникальные токены
3. Обновляем self.term2num_docs - массив, в котором для каждого токена хранится число того, в скольких уникальных документах этот токен встречается. Токен `<UNK>` игнорируем.

### idf
Считаем логарифмированный inverse document frequency для документа

$$
idf = -log \frac {n_t + 1} {N}
$$
где $n_t$ - в скольких документах встречается токен, $N$ - число различных документов. За эти параметры у нас отвечают параметры `self.term2num_docs` и `self.num_docs`. (единицы мы добавляем, чтобы избежа

### predict
1. Получаем набор документов, на выходе генерируем numpy матрицу tf-idf размера len(docs) x len(vocabulary)
2. Токенизируем каждый текст
3. Для каждого уникального токена в тексте (кроме `<UNK>`) считаем tf - как часто данный токен встречается во всем тексте (с учетом дублей и `<UNK>` токена!)
4. Для каждого токена считаем idf из одноименной функции
5. Заполняем соответствующие элементы массива
6. Нормализуем вектор

In [11]:
class TFIDF:
    def __init__(self, tokenizer: Tokenizer) -> None:
        """
        Инициализация TFIDF.

        :param tokenizer: токенизатор для преобразования текста в токены
        :param default_idf: значение IDF для неизвестных токенов
        """
        self.tokenizer = tokenizer
        self.num_docs = 0
        self.term2num_docs = [0 for _ in tokenizer.word2idx]  # для подсчёта IDF

    @property
    def vocab_size(self) -> int:
        """
        Возвращает размер словаря.

        :return: размер словаря
        """
        return len(self.tokenizer)
    
    def add_doc(self, doc: str) -> None:
        """
        Добавляет документ в модель TFIDF.
        1. Увеличиваем счетчик числа документов
        2. Токенизируем текст
        3. Для всех уникальных токенов обновляем self.term2numdocs для подсчета IDF

        :param doc: документ для добавления
        """
        # raise NotImplementedError()
        self.num_docs += 1
        tokens = set(tokenize(doc))
        ixs = self.tokenizer.encode(' '.join(tokens))
        for ix in ixs:
            # пропускаем <UNK>
            if ix == self.tokenizer.unk_token_id:
                continue    
            self.term2num_docs[ix] += 1
        

    def fit(self, docs: List[str]) -> None:
        """
        Обучает модель TFIDF на корпусе docs.

        :param docs: корпус для обучения
        """
        for doc in docs:
            self.add_doc(doc)

    def predict(self, docs: List[str]) -> np.ndarray:
        """
        Предсказывает TFIDF значения для списка документов.
        1. Создаем np.array размерности (len(docs), vocab_size)
        2. Для каждого документа
            а. Токенизируем его
            б. Считаем tf для каждого токена кроме unk
            в. Считаем idf для каждого токена кроме unk
            г. Заполняем соответствующее значение в матрице
        3. Нормализуем матрицу по размерности словаря (по строкам). При нормализации, 
        чтобы избежать деления на 0 делите не на норму ветктора, а на норму вектора + 1e-5

        :param docs: список документов
        :return: матрица TFIDF значений
        """
        # создаем numpy массив нулей размера len(docs) на размерность словаря
        result = np.zeros(shape = (len(docs), self.vocab_size))
        # для каждого документа будем считать его вектор tf-idf
        for doc_idx, document_text in enumerate(docs):
            # токенизируем документ
            tokens = tokenize(document_text)
            # считаем tf для каждого токена
            tf = Counter(tokens)
            # для каждого токена считаем его tf-idf
            for token in tokens:
                token_id = self.tokenizer.encode_word(token)
                if token_id == self.tokenizer.unk_token_id:
                    continue
                tf_idf = tf[token] * self.idf(token_id)
                result[doc_idx, token_id] = tf_idf
        norm = np.linalg.norm(result, axis=1)[:, None]
        # нормализуем документ
        result = result / (norm + 1e-5)
        return result

    def idf(self, token_id: int) -> float:
        """
        Вычисляет IDF (обратную частоту документа) для термина.

        :param term: термин
        :return: IDF значение
        """
        # raise NotImplementedError()
        return np.log(self.num_docs / (self.term2num_docs[token_id] + 1))


In [12]:
corpus = [
    "hello there", "my favourite frut is", "i love bananas", "hello mama", "I need to eat",
    "how can I get to the studio", "bottom gear"
]
tokenizer = Tokenizer(corpus, min_count=1)
tfidf = TFIDF(tokenizer)
tfidf.fit(corpus)

assert not np.any(tfidf.predict(["all tokens abscent from vocab should be zeros vector"]))
reference = np.zeros((1, len(tfidf.tokenizer)))
reference[0, tfidf.tokenizer.encode_word("hello")] = 2 / 3 * -np.log(3/7)
reference[0, tfidf.tokenizer.encode_word("mama")] = 1 / 3 * -np.log(2/7)
reference /= 0.7024715222440031
assert np.allclose(tfidf.predict(["hello hello mama"]), reference)
print("test_tfidf OK")


Считаем слова:   0%|          | 0/7 [00:00<?, ?it/s]

Строим словарь:   0%|          | 0/20 [00:00<?, ?it/s]

test_tfidf OK


## Классификация с помощью TF-IDF - 10 баллов
В этом задании предлагается обучить с помощью полученного векторизатора TF-IDF логистическую регресиию.
Для этого мы возьмем корпус IMDB - отзывы фильмов. Это задача бинарной классификации, в которой нужно определить - положительный отзыв или отрицательный.

Задача будет решаться следующими этапами:
1. Загружаем текстовый корпус, обучаем словарь и TFIDF
2. Векторизуем корпус текстов
3. По векторизованному корпусу и меткам текстов обучаем логистическую регрессию, смотрим на качество на тесте

In [14]:
# загружаем датасет
imdb = load_dataset("stanfordnlp/imdb")
train_dataset = [imdb["train"][i] for i in range(10_000, 15_000)]
test_dataset = [imdb["test"][i] for i in range(10_000, 15_000)]

print("Label")
print(train_dataset[0]["label"])
print("Text")
print(train_dataset[0]["text"])

Label
0
Text
Someone actually gave this movie 2 stars. There's a very high chance they need immediate professional help as anyone who doesn't spend 30 seconds to see if you can award no stars is quite literally scary.<br /><br />This film is ... well ... I guess it's pretty much some kind of attempt at a horrible porn / snuff movie with no porn or no real horrible bits (apart from the acting, plot, story, sets, dialogue and sound). I wrongly assumed it was about zombies. <br /><br />Watching it is actually quite scary in fairness; you're terrified someone will come over and you'll never be able to describe what it is and they'll go away thinking you're a freak that watches home-made amateur torture videos or something along those lines. <br /><br />I'm so taken aback I'm writing this review on my mobile so I don't forget to attempt to bring the rating down further than the current 1.6 to save others from the same horrible fate that I just suffered. <br /><br />I worst film I've ever se

In [15]:
train_texts = [sample["text"] for sample in train_dataset]
test_texts = [sample["text"] for sample in test_dataset]

# Создаем токенизатор, min_count можете взять на свое усмотрение, но лучше брать в районе 5-10;

# 4. Обучаем по ним LogisticRegression, accuracy на тесте должен быть 0.75+


# 1. Токенизатор обучаем по входным текстам
tokenizer = Tokenizer(train_texts, min_count=5)
# 2. Обучаем по этим же текстам с этим токенайзером TFIDF
tfidf = TFIDF(tokenizer)
tfidf.fit(train_texts)

# 3. превращаем тексты в numpy матрицы
X_train = tfidf.predict(train_texts)
X_test = tfidf.predict(test_texts)

Y_train = np.array([sample["label"] for sample in train_dataset])
Y_test = np.array([sample["label"] for sample in test_dataset])

# Обучаем логистическую регрессию и смотрим на качество
clf = LogisticRegression()
clf.fit(X_train, Y_train)
prediction = clf.predict(X_test)

accuracy = (prediction == Y_test).sum() / Y_test.shape[0]
print(f"accuracy = {accuracy}")


Считаем слова:   0%|          | 0/5000 [00:00<?, ?it/s]

Строим словарь:   0%|          | 0/34695 [00:00<?, ?it/s]

accuracy = 0.8432


Теперь обучим логистическую регрессию со второй TF-IDF моделью и сравним результаты. Для этого воспользуйтесь классом [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) из sklearn. Результаты должны быть похожи на вашу реализацию.

In [19]:
train_texts = [sample["text"] for sample in train_dataset]
test_texts = [sample["text"] for sample in test_dataset]
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=5)

X_train = tfidf.fit_transform(train_texts)
X_test = tfidf.transform(test_texts)

Y_train = np.array([sample["label"] for sample in train_dataset])
Y_test = np.array([sample["label"] for sample in test_dataset])

# Обучаем логистическую регрессию и смотрим на качество
clf = LogisticRegression()
clf.fit(X_train, Y_train)
prediction = clf.predict(X_test)

accuracy = (prediction == Y_test).sum() / Y_test.shape[0]
print(f"accuracy = {accuracy}")

/Users/igorchebuniaev/Library/Caches/pypoetry/virtualenvs/gmvae-giXL_VnD-py3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


accuracy = 0.8568


## $n$-граммные языковые модели (15 баллов)

### Расширяем Токенайзер (3 балла)

Перед созданием языковой модели, расширим токенизационный класс. Добавим два флага в сигнатуру метода `encode`, чтобы управлять добавлением служебных токенов во время токенизации. Существующий метод `decode` уже пропускает `<PAD>` токен, добавим флаг `skip_special_tokens` для пропуска всех специальных токенов.

In [20]:
class BoSTokenizerEoS(Tokenizer):
    def encode(self, text: str, add_bos: bool = True, add_eos: bool = False) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :param add_bos: добавление begin-of-sentence токена в начало
        :param add_eos: добавление end-of-sentence токена в конец
        :return: набор индексов токенов
        """
        tokens = tokenize(text)
        tokens = [normalize(token) for token in tokens]
        if add_bos:
            tokens = ["<BOS>"] + tokens
        if add_eos:
            tokens = tokens + ["<EOS>"]
        return [self.word2idx.get(token, self.unk_token_id) for token in tokens]

    def decode(self, input_ids: List[int], skip_special_tokens: bool = True) -> str:
        """
        Декодирование набора индексов в текст. Вставляет пробел между декодированнми токенами.

        :param input_ids: набор индексов токенов
        :param skip_special_tokens: пропуск специальных токенов во время декодирования.
        :return: текст
        """
        if skip_special_tokens:
            input_ids = [idx for idx in input_ids if idx not in range(4)]
        return " ".join([self.idx2word[idx] for idx in input_ids])


In [21]:
corpus = ["Hello, world!", "I love Python!", "Hello, Python", "Hello there <EOS>"]
tokenizer = BoSTokenizerEoS(corpus, min_count=1)
# tokenizer.encode("hello world", add_bos=True, add_eos=True)

Считаем слова:   0%|          | 0/4 [00:00<?, ?it/s]

Строим словарь:   0%|          | 0/11 [00:00<?, ?it/s]

In [24]:

corpus = ["Hello, world!", "I love Python!", "Hello, Python", "Hello there <EOS>"]
tokenizer = BoSTokenizerEoS(corpus, min_count=1)
# референсные индексы для world справа изменены (8 -> 6), обсуждение тут: https://t.me/c/2282620518/157/183
assert tokenizer.encode("hello world", add_bos=True, add_eos=True) == [1, 4, 6, 2]
assert tokenizer.encode("hello world", add_bos=False, add_eos=False) == [4, 6]
print("test_bos_tokenizer OK")


Считаем слова:   0%|          | 0/4 [00:00<?, ?it/s]

Строим словарь:   0%|          | 0/11 [00:00<?, ?it/s]

test_bos_tokenizer OK


### Создаём NGram Модель (12 баллов)

Создайте класс `NGramLanguageModel` для построения n-граммной языковой модели. В этом задании вы можете как опираться на предложенную структуру модели, так и сделать свою имплементацию.

Построение модели:
   - Создайте метод `_build_model`, который принимает список текстов `texts` и обновляет частоты n-грамм.
   - Для каждого текста:
     - Токенизируйте текст и добавьте токен `"<EOS>"` в конец.
     - Для каждого токена:
       - Определите префикс длиной `n-1`.
       - Обновите частоты n-грамм и частоты префиксов.

Генерация следующего токена:
   - Создайте метод `generate_next_token`, который принимает префикс `prefix` и возвращает следующий токен.
   - Преобразуйте префикс в кортеж.
   - Получите распределение частот для префикса.
   - Если распределение пустое, верните токен `"<UNK>"`.
   - Верните токен с наибольшей частотой.
   - Если включен флаг sample возьмите токен пропорционально встречаемости

Автодополнение текста:
   - Создайте метод `autocomplete`, который принимает текст `text` и максимальную длину `max_len`, и возвращает завершенный текст.
   - Токенизируйте текст.
   - Пока длина токенов меньше `max_len`:
     - Определите префикс длиной `n-1`.
     - Сгенерируйте следующий токен.
     - Добавьте токен в список токенов.
     - Если токен равен `"<EOS>"`, завершите генерацию.
   - Декодируйте и верните текст.

In [25]:
class NGramLanguageModel:
    def __init__(self, n: int, tokenizer: BoSTokenizerEoS, texts: List[str]):
        """
        Создание n-граммной языковой модели.

        :param n: порядок n-грамм
        :param vocabulary: словарь
        """
        assert n >= 2
        self.n = n
        self.tokenizer = tokenizer
        # Словарь, ключ которого - префикс
        # значение - Counter(), в котором ключ это продолжение n-граммы, 
        # а значение - частота встречи этого продолжения
        self.frequencies = defaultdict(lambda: Counter())  # частота n-грамм
        self._build_model(texts)

    def _build_model(self, texts: List[str]):
        """
        Построение модели на основе списка текстов.

        :param texts: список текстов
        """
        # Токенизируйте текст и добавьте токен `"<EOS>"` в конец.
        for text in texts:
            embedding = self.tokenizer.encode(text, add_eos=True)
            # Для каждого токена:
            # Определите префикс длиной `n-1`.
            for i in range(len(embedding) - self.n + 1):
                prefix = tuple(embedding[i:i + self.n - 1])
                current_token = embedding[i + self.n - 1]
                # Обновите частоты n-грамм и частоты префиксов.
                self.frequencies[prefix].update([current_token])
            # Обновите частоты n-грамм и частоты префиксов.


    def generate_next_token(self, prefix: List[int], sample: bool = False) -> int:
        """
        Генерация следующего токена по префиксу.

        :param prefix: префикс
        :param sample: используем ли мы сэмплинг в генерации
        :return: следующий токен
        """
        # Преобразуйте префикс в кортеж.
        prefix = tuple(prefix)
        # Получите распределение частот для префикса.
        distribution = self.frequencies[prefix]
        # Если распределение пустое, верните токен `"<UNK>"`.
        if not distribution:
            return self.tokenizer.unk_token_id
        # Верните токен с наибольшей частотой.
        if not sample:
            return distribution.most_common(1)[0][0]
        # Если включен флаг sample возьмите токен пропорционально встречаемости
        tokens = list(distribution.keys())
        weights = list(distribution.values())
        return random.choices(tokens, weights=weights)[0]
        
        

    def autocomplete(self, text: str, max_len: int = 32, 
                     skip_special_tokens: bool = True, sample: bool = False) -> str:
        """
        Автоматическое дополнение текста.

        :param text: текст
        :param max_len: максимальная длина текста
        :param skip_special_tokens: пропуск специальных токенов во время декодирования.
        :return: завершенный текст
        """
        # Токенизируйте текст.
        tokens = self.tokenizer.encode(text, add_bos=True, add_eos=False)
        # Пока длина токенов меньше `max_len`:
        while len(tokens) < max_len:
            #     Определите префикс длиной `n-1`.
            prefix = tokens[-self.n + 1:]
            #     Сгенерируйте следующий токен.
            next_token = self.generate_next_token(prefix=prefix, sample=sample)
            #     Добавьте токен в список токенов.
            tokens.append(next_token)
            #     Если токен равен `"<EOS>"`, завершите генерацию.
            if next_token == self.tokenizer.word2idx["<EOS>"]:
                break
        # Декодируйте и верните текст.
        return self.tokenizer.decode(input_ids=tokens, skip_special_tokens=skip_special_tokens)

corpus = ["Hello, world!", "I love Python!", "Hello, Python"]
tokenizer = BoSTokenizerEoS(corpus, min_count=1)
ngram_lm = NGramLanguageModel(2, tokenizer, corpus)

assert ngram_lm.autocomplete("Hello, Python", max_len=10) == "hello , python !"
assert ngram_lm.autocomplete("Hello, Python", max_len=10, skip_special_tokens=False) == "<BOS> hello , python ! <EOS>"

imdb = load_dataset("stanfordnlp/imdb")
train_texts = [imdb["train"][i]["text"] for i in range(13_000, 15_000)]
tokenizer = BoSTokenizerEoS(texts=train_texts, min_count=3)
ngram_lm = NGramLanguageModel(3, tokenizer, train_texts)
print("Продолжение фразы `the movie was`")
random.seed(1)
print(ngram_lm.autocomplete("the movie was", sample=True))
print("test_ngram_model OK")


Считаем слова:   0%|          | 0/3 [00:00<?, ?it/s]

Строим словарь:   0%|          | 0/7 [00:00<?, ?it/s]

Считаем слова:   0%|          | 0/2000 [00:00<?, ?it/s]

Строим словарь:   0%|          | 0/20956 [00:00<?, ?it/s]

Продолжение фразы `the movie was`
the movi wa releas the rosenstrass protest in berlin and marri the woman listen in silenc , when they re obvious be compar to his work. < br / > <
test_ngram_model OK
